In [2]:
from pathlib import Path
import pandas as pd
import glob
import numpy as np

import os, glob, json, math
from collections import defaultdict, Counter

import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

# Для графиков: по правилам — только matplotlib, один график на ячейку, без явных цветов.

BASE = Path("data")                # если ты запускаешь из корня проекта
TX_DIR = BASE / "transactions"     # папка из твоего скрина

clients = pd.read_csv(BASE / "clients.csv")
clients.columns = [c.strip() for c in clients.columns]

# 1) Транзакции (покупки)
tx_files = sorted(glob.glob(str(TX_DIR / "client_*_transactions_3m.csv")))
tx_frames = [pd.read_csv(p, sep=None, engine="python") for p in tx_files]  # авто-определение разделителя
tx = pd.concat(tx_frames, ignore_index=True) if tx_frames else pd.DataFrame()
tx.columns = [c.strip() for c in tx.columns]

# 2) Переводы (in/out)
tr_files = sorted(glob.glob(str(TX_DIR / "client_*_transfers_3m.csv")))
tr_frames = [pd.read_csv(p, sep=None, engine="python") for p in tr_files]
tr = pd.concat(tr_frames, ignore_index=True) if tr_frames else pd.DataFrame()
tr.columns = [c.strip() for c in tr.columns]

tx["date"] = pd.to_datetime(tx.get("date"), errors="coerce")
tr["date"] = pd.to_datetime(tr.get("date"), errors="coerce")

for df in (tx, tr):
    if "amount" in df.columns:
        df["amount"] = pd.to_numeric(df["amount"], errors="coerce").fillna(0.0)
    for col in ("category","currency","type","direction"):
        if col in df.columns:
            df[col] = df[col].fillna("").astype(str)

# Валютные курсы (оффлайн дефолты). При наличии проекта можно читать exchange_rates.json.
rates = {"KZT":1.0, "USD":500.0, "EUR":540.0, "RUB":5.0}
def to_kzt(amount, currency):
    try:
        r = rates.get(str(currency).upper(), 1.0)
        return float(amount) * float(r)
    except Exception:
        return 0.0

tx["amount"] = pd.to_numeric(tx.get("amount", 0), errors="coerce").fillna(0.0)
tx["amount_kzt"] = tx.apply(lambda r: to_kzt(r["amount"], r["currency"]), axis=1)

# tx
tx["date"] = pd.to_datetime(tx.get("date"), errors="coerce")
for col in ("category","currency","client_code"):
    if col in tx.columns:
        tx[col] = tx[col].fillna("").astype(str)
tx["amount"] = pd.to_numeric(tx.get("amount", 0.0), errors="coerce").fillna(0.0)
tx["amount_kzt"] = tx.apply(lambda r: to_kzt(r["amount"], r["currency"]), axis=1)
tx["ym"] = tx["date"].dt.to_period("M")

tr["date"] = pd.to_datetime(tr.get("date"), errors="coerce")
for col in ("type","direction","currency","client_code"):
    if col in tr.columns:
        tr[col] = tr[col].fillna("").astype(str)
tr["amount"] = pd.to_numeric(tr.get("amount", 0.0), errors="coerce").fillna(0.0)
tr["amount_kzt"] = tr.apply(lambda r: to_kzt(r["amount"], r["currency"]), axis=1)
tr["ym"] = tr["date"].dt.to_period("M")


In [3]:
# === DROP-IN FIX: нормализация колонок и ключа ===
import re

def normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    # нижний регистр, срез пробелов, пробелы и «экзотика» → подчеркивания
    mapping = {c: re.sub(r'[^a-z0-9_]', '_', c.strip().lower()) for c in df.columns}
    return df.rename(columns=mapping)

def ensure_client_code(df: pd.DataFrame, df_name="df") -> pd.DataFrame:
    if "client_code" in df.columns:
        return df
    # пытаемся угадать поле клиента
    cand = [c for c in df.columns if "client" in c and ("code" in c or "id" in c)]
    if cand:
        return df.rename(columns={cand[0]: "client_code"})
    raise KeyError(f"{df_name}: не нашли колонку client_code (после нормализации: {list(df.columns)})")

# 1) нормализуем названия колонок
clients = normalize_cols(clients)
tx      = normalize_cols(tx)
tr      = normalize_cols(tr)

# 2) убеждаемся, что есть client_code (подхватываем альтернативные названия)
clients = ensure_client_code(clients, "clients")
if not tx.empty:
    tx = ensure_client_code(tx, "tx")
if not tr.empty:
    tr = ensure_client_code(tr, "tr")

# 3) приводим ключ к строке везде (чтобы не было несоответствия int vs str)
clients["client_code"] = clients["client_code"].astype(str)
if "client_code" in tx.columns:
    tx["client_code"] = tx["client_code"].astype(str)
if "client_code" in tr.columns:
    tr["client_code"] = tr["client_code"].astype(str)

# 4) на всякий случай проверим наличие критичных полей
for df, name in [(tx,"tx"), (tr,"tr")]:
    if df.empty: 
        continue
    for c in ["date","amount","currency"]:
        if c not in df.columns:
            print(f"[WARN] {name}: нет колонки {c} — проверь входные файлы")

print("OK cols:", clients.columns.tolist()[:8], "...", " | tx:", list(tx.columns)[:8], " | tr:", list(tr.columns)[:8])


tx_groups = {str(k): g for k, g in (tx.groupby("client_code") if not tx.empty else [])}
tr_groups = {str(k): g for k, g in (tr.groupby("client_code") if not tr.empty else [])}

ONLINE = {"Едим дома","Смотрим дома","Играем дома"}
TRAVEL = {"Путешествия","Отели","Такси"}

def monthly_totals(df, col="amount_kzt"):
    if df.empty: return {}
    tmp = df.copy()
    tmp["ym"] = tmp["date"].dt.to_period("M")
    s = tmp.groupby("ym")[col].sum()
    return {str(k): float(v) for k, v in s.items()}

def get_num_from_row(row, *aliases, default=0.0):
    """Достаёт число по одному из возможных имён колонки (безопасно).
       Чистит пробелы/нецифры, поддерживает ',' как десятичный разделитель."""
    for key in aliases:
        if key in row and pd.notna(row[key]):
            s = str(row[key]).replace('\xa0', '').replace(' ', '')
            s = re.sub(r'[^0-9,.\-]', '', s).replace(',', '.')
            try:
                return float(s)
            except Exception:
                pass
    return float(default)

rows = []
for _, p in clients.iterrows():
    code = str(p.get("client_code"))
    df_tx = tx_groups.get(code, pd.DataFrame(columns=tx.columns))
    df_tr = tr_groups.get(code, pd.DataFrame(columns=tr.columns))

    # Покупки
    if df_tx.empty:
        spend_cat = {}; total_spend = 0.0
        fx_share_tx = 0.0
        months_tx = {}
    else:
        spend_cat = df_tx.groupby("category")["amount_kzt"].sum().to_dict()
        total_spend = sum(spend_cat.values())
        tot = df_tx["amount_kzt"].sum()
        fx_amt = df_tx.loc[df_tx["currency"].str.upper().ne("KZT"), "amount_kzt"].sum()
        fx_share_tx = (fx_amt/tot) if tot>0 else 0.0
        months_tx = monthly_totals(df_tx)

    online_share = (sum(spend_cat.get(c,0.0) for c in ONLINE)/total_spend) if total_spend>0 else 0.0
    travel_share = (sum(spend_cat.get(c,0.0) for c in TRAVEL)/total_spend) if total_spend>0 else 0.0

    # Переводы
    if df_tr.empty:
        in_sum = out_sum = 0.0
        salary_in = stipend_in = cashback_in = refund_in = 0.0
        p2p_out = card_out = atm_withdrawal = utilities_out = 0.0
        loan_payment_out = cc_repayment_out = installment_payment_out = 0.0
        invest_in = invest_out = deposit_topup_out = 0.0
        gold_buy_out = gold_sell_in = 0.0
        months_tr_in = months_tr_out = {}
    else:
        in_sum  = df_tr.loc[df_tr["direction"].eq("in"), "amount_kzt"].sum()
        out_sum = df_tr.loc[df_tr["direction"].eq("out"), "amount_kzt"].sum()
        # ключевые типы
        def s(tp): return df_tr.loc[df_tr["type"].eq(tp), "amount_kzt"].sum()
        salary_in = s("salary_in"); stipend_in = s("stipend_in")
        cashback_in = s("cashback_in"); refund_in = s("refund_in")
        p2p_out = s("p2p_out"); card_out = s("card_out"); atm_withdrawal = s("atm_withdrawal")
        utilities_out = s("utilities_out"); loan_payment_out = s("loan_payment_out")
        cc_repayment_out = s("cc_repayment_out"); installment_payment_out = s("installment_payment_out")
        invest_in = s("invest_in"); invest_out = s("invest_out")
        deposit_topup_out = s("deposit_topup_out")
        gold_buy_out = s("gold_buy_out"); gold_sell_in = s("gold_sell_in")

        months_tr_in  = monthly_totals(df_tr.loc[df_tr["direction"].eq("in")])
        months_tr_out = monthly_totals(df_tr.loc[df_tr["direction"].eq("out")])
    
    # summary row
    rows.append({
        "client_code": code,
        "status": str(p.get("status","")),
        "age": float(p.get("age", np.nan)) if pd.notna(p.get("age", np.nan)) else np.nan,
        "avg_balance": float(p.get("avg_monthly_balance_kzt", 0.0) or 0.0),

        "total_spend_3m": float(total_spend),
        "share_online": float(online_share),
        "share_travel": float(travel_share),
        "months_seen_tx": int(len(months_tx)),
        "month_spend_mean": float(np.mean(list(months_tx.values()))) if months_tx else 0.0,
        "month_spend_std":  float(np.std(list(months_tx.values()))) if months_tx else 0.0,

        "transfers_in_3m": float(in_sum),
        "transfers_out_3m": float(out_sum),
        "net_flow_3m": float(in_sum - out_sum),

        "salary_in": float(salary_in),
        "stipend_in": float(stipend_in),
        "cashback_in": float(cashback_in),
        "refund_in": float(refund_in),

        "p2p_out": float(p2p_out),
        "card_out": float(card_out),
        "atm_withdrawal": float(atm_withdrawal),
        "utilities_out": float(utilities_out),
        "loan_payment_out": float(loan_payment_out),
    })

feat = pd.DataFrame(rows)
feat = feat.merge(clients[["client_code","city"]], on="client_code", how="left")
feat.head(10)

OK cols: ['client_code', 'name', 'status', 'age', 'city', 'avg_monthly_balance_kzt'] ...  | tx: ['client_code', 'name', 'product', 'status', 'city', 'date', 'category', 'amount']  | tr: ['client_code', 'name', 'product', 'status', 'city', 'date', 'type', 'direction']


,client_code,status,age,avg_balance,total_spend_3m,share_online,share_travel,months_seen_tx,month_spend_mean,month_spend_std,transfers_in_3m,transfers_out_3m,net_flow_3m,salary_in,stipend_in,cashback_in,refund_in,p2p_out,card_out,atm_withdrawal,utilities_out,loan_payment_out,city
0,1,Зарплатный клиент,29.0,92643.0,2626914.27,0.184009,0.244409,3,8.756381e+05,172513.468114,1875090.64,6722190.40,-4847099.76,1446421.73,0.00,138273.34,114471.50,1274129.26,3829951.76,617946.71,492152.41,508010.26,Алматы
1,2,Премиальный клиент,41.0,1577073.0,2623272.32,0.180276,0.166927,3,8.744241e+05,89349.195422,1646273.33,6636050.47,-4989777.14,1278082.01,0.00,107537.19,55248.95,1305706.44,3652000.64,775789.21,396920.77,505633.41,Астана
2,3,Студент,22.0,63116.0,2272999.63,0.198575,0.320537,3,7.576665e+05,251780.894628,690885.17,3526821.01,-2835935.84,0.00,124207.58,187247.61,60804.20,496377.33,1315353.95,759624.87,395640.68,559824.18,Алматы
3,4,Зарплатный клиент,36.0,83351.0,2540993.42,0.177756,0.241687,3,8.469978e+05,165135.314667,1786681.16,6103937.34,-4317256.18,1372775.52,0.00,107111.63,61478.54,1170717.90,3312986.71,634755.85,453960.22,531516.66,Караганда
4,5,Премиальный клиент,45.0,1336536.0,2703671.00,0.199707,0.237067,3,9.012237e+05,182854.492842,1746521.40,6200126.04,-4453604.64,1262119.01,0.00,126590.18,74928.97,1261804.77,3344845.96,590493.11,454938.05,548044.15,Алматы
5,6,Стандартный клиент,34.0,131929.0,2670256.87,0.159337,0.220022,3,8.900856e+05,131759.369281,1828401.34,6432809.25,-4604407.91,1378299.83,0.00,109520.75,60443.57,1331551.75,3619748.11,629865.51,370190.08,481453.80,Шымкент
6,7,Премиальный клиент,48.0,4040997.0,2701821.89,0.178182,0.087017,3,9.006073e+05,162336.753215,1798855.62,8422453.40,-6623597.78,1197030.77,0.00,182859.58,132023.71,1572706.70,4110840.27,1193728.17,557691.12,987487.14,Алматы
7,8,Зарплатный клиент,33.0,1058403.0,3215125.84,0.161322,0.073257,3,1.071709e+06,37334.055039,2044952.12,8494299.18,-6449347.06,1349955.04,0.00,266820.85,109128.90,1730533.82,4048995.40,1162883.30,609707.88,942178.78,Астана
8,9,Премиальный клиент,55.0,3084180.0,2823888.90,0.163584,0.079068,3,9.412963e+05,101118.252239,1783301.11,8565700.19,-6782399.08,1205500.19,0.00,188381.32,92987.82,1825147.95,4138180.77,1115664.13,551803.57,934903.77,Алматы
9,10,Зарплатный клиент,38.0,1277325.0,2919534.74,0.141674,0.068140,3,9.731782e+05,38041.966619,1978310.07,8538828.01,-6560517.94,1293311.35,0.00,184539.23,208027.80,1774833.09,4160053.26,1251711.25,562785.97,789444.44,Усть-Каменогорск


In [4]:
# Аудит текущей витрины feat (если есть файл — можно читать его)
import pandas as pd
import numpy as np
from pathlib import Path


# НЕ трогаем служебные
ID_COLS  = ["client_code"]
CAT_COLS = ["status","city"]
TARGET_COL = "teacher_best"  # подольём из teacher_df чуть ниже

# добавим teacher label
feat_in = feat.copy()

num_cols = feat_in.select_dtypes(include=[np.number]).columns.tolist()
# исключим явные таргеты/выводные
num_cols = [c for c in num_cols if c not in []]  # оставим все числовые из витрины

audit = []
for c in num_cols:
    s = feat_in[c]
    nnz = (s != 0).sum()
    nonzero_rate = nnz / len(s)
    nunique = s.nunique(dropna=False)
    nan_rate = s.isna().mean()
    var = s.var(ddof=1)
    audit.append(dict(
        feature=c, nunique=int(nunique), nonzero_rate=float(nonzero_rate),
        nan_rate=float(nan_rate), mean=float(s.mean()), std=float(s.std(ddof=1) or 0.0),
        var=float(var or 0.0)
    ))
audit_df = pd.DataFrame(audit).sort_values(["nonzero_rate","nunique"], ascending=[True, True]).reset_index(drop=True)
audit_df.to_csv("outputs/feature_audit.csv", index=False, encoding="utf-8")
audit_df.head(20)


,feature,nunique,nonzero_rate,nan_rate,mean,std,var
0,stipend_in,4,0.050000,0.0,5.681137e+03,2.507565e+04,6.287882e+08
1,salary_in,58,0.950000,0.0,1.396528e+06,5.552216e+05,3.082711e+11
2,months_seen_tx,2,0.966667,0.0,2.900000e+00,5.430610e-01,2.949153e-01
3,total_spend_3m,59,0.966667,0.0,2.463356e+06,5.914028e+05,3.497573e+11
4,share_online,59,0.966667,0.0,1.914893e-01,5.802374e-02,3.366754e-03
5,share_travel,59,0.966667,0.0,1.341004e-01,6.541305e-02,4.278867e-03
6,month_spend_mean,59,0.966667,0.0,8.211187e+05,1.971343e+05,3.886192e+10
7,month_spend_std,59,0.966667,0.0,7.136998e+04,6.874770e+04,4.726246e+09
8,age,33,1.000000,0.0,3.751667e+01,9.145698e+00,8.364379e+01
9,avg_balance,60,1.000000,0.0,1.094079e+06,1.417719e+06,2.009927e+12


In [5]:
# Пороги можно подкрутить
MIN_NONZERO_RATE = 0.02     # <2% наблюдений ≈ квазисобытие — кандидаты на дроп
MIN_NUNIQUE = 2             # константы на выброс
MIN_VARIANCE = 1e-9         # почти константы

rule_drop = set()
for _, r in audit_df.iterrows():
    if r["nunique"] < MIN_NUNIQUE or r["var"] < MIN_VARIANCE:
        rule_drop.add(r["feature"])
    elif r["nonzero_rate"] < MIN_NONZERO_RATE:
        # не дропаем сразу — возможно, редкий, но важный признак → проверим MI на следующем шаге
        pass

# Защитим критичные признаки руками (блендер может решить оставить)
WHITELIST = set(["age","avg_balance","total_spend_3m","net_flow_3m","fx_share_tx","share_travel","share_online"])
rule_drop = {f for f in rule_drop if f not in WHITELIST}
len(rule_drop), list(sorted(rule_drop))[:10]


(0, [])

In [6]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.impute import SimpleImputer

# Целевой столбец
y = feat_in[TARGET_COL].astype(str).fillna("NA")

# MI считаем по числовым признакам, с простейшим имьютером
X_num = feat_in[num_cols].copy()
imp = SimpleImputer(strategy="median")
X_num_imp = imp.fit_transform(X_num)

mi = mutual_info_classif(X_num_imp, y, discrete_features=False, random_state=42)
mi_series = pd.Series(mi, index=num_cols, name="mi")

# низкая MI — кандидаты на дроп (если ещё и редкие)
MI_MIN = 0.003
rare_feats = set(audit_df.loc[audit_df["nonzero_rate"] < MIN_NONZERO_RATE, "feature"])
mi_drop = set(mi_series.index[(mi_series < MI_MIN) & (mi_series.index.isin(rare_feats))])

# Правило №3: сильная корреляция — оставляем один из пары (у кого MI больше)
CORR_MAX = 0.95
corr = pd.DataFrame(X_num_imp, columns=num_cols).corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

corr_drop = set()
for col in upper.columns:
    hits = [row for row in upper.index if (not np.isnan(upper.loc[row, col])) and (upper.loc[row, col] > CORR_MAX)]
    for row in hits:
        # дропнем тот, у кого MI меньше (если равны — у кого дисперсия меньше)
        keep = col if mi_series[col] >= mi_series[row] else row
        drop = row if keep == col else col
        corr_drop.add(drop)

to_drop = (rule_drop | mi_drop | corr_drop) - WHITELIST
len(to_drop), list(sorted(to_drop))[:15]


KeyError: 'teacher_best'

In [ ]:
# Глобальные пороги (подбираются по данным; можно менять квантили)
import numpy as np
import pandas as pd

TRAVEL = {"Путешествия","Отели","Такси"}
ONLINE = {"Едим дома","Смотрим дома","Играем дома"}

# пер-клиент суммы
totals_by_client = tx.groupby("client_code")["amount_kzt"].sum()
travel_by_client = tx[tx["category"].isin(TRAVEL)].groupby("client_code")["amount_kzt"].sum()
online_by_client = tx[tx["category"].isin(ONLINE)].groupby("client_code")["amount_kzt"].sum()
fx_by_client = tx.loc[tx["currency"].str.upper().ne("KZT")].groupby("client_code")["amount_kzt"].sum()

share_travel = (travel_by_client / totals_by_client).fillna(0.0)
share_online = (online_by_client / totals_by_client).fillna(0.0)

# переводы/снятия (для премиальной)
tr_out = tr[tr["direction"].eq("out")]
prem_out_by_client = (
    tr_out.loc[tr_out["type"].isin(["atm_withdrawal","p2p_out","card_out"])]
    .groupby("client_code")["amount_kzt"].sum()
)

# «свободный остаток»: возьмём из витрины feat (или прикинем заново при отсутствии)
if "free_balance" not in locals():
    # оценка free_balance = avg_balance - медианная месячная трата (>=0)
    free_balance_map = {}
    for code, dfc in tx.groupby("client_code"):
        m = dfc.assign(ym=dfc["date"].dt.to_period("M")).groupby("ym")["amount_kzt"].sum()
        med_spend = float(m.median()) if len(m) else 0.0
        avg_bal = float(clients.set_index("client_code").loc[code, "avg_monthly_balance_kzt"])
        free_balance_map[str(code)] = max(0.0, avg_bal - med_spend)
    free_balance = pd.Series(free_balance_map, name="free_balance")
else:
    free_balance = feat.set_index("client_code")["avg_balance"] - feat.set_index("client_code")["month_spend_mean"]

# КВАНТИЛИ
thr = {
    "travel_sum":     float(travel_by_client.quantile(0.70)) if not travel_by_client.empty else 100_000,
    "travel_share":   float(share_travel.quantile(0.70)),
    "online_share":   float(share_online.quantile(0.70)),
    "prem_out_sum":   float(prem_out_by_client.quantile(0.70)) if not prem_out_by_client.empty else 200_000,
    "free_balance":   float(free_balance.quantile(0.60)),
}
thr


In [ ]:
TRAVEL = {"Путешествия","Отели","Такси"}
ONLINE = {"Едим дома","Смотрим дома","Играем дома"}
PREMIUM_4 = {"Ювелирные украшения","Косметика и Парфюмерия","Кафе и рестораны"}

# гипотезы капов (на месяц) — можно двинуть числа
CC_TOP3_CAP_SUM   = 300_000     # до этой суммы в месяц 10% на топ-3
CC_ONLINE_CAP_SUM = 100_000     # до этой суммы в месяц 10% на онлайн-сервисы
PREMIUM_CASHBACK_CAP_PER_M = 100_000  # из ТЗ (кешбэк 100k/мес)
PREMIUM_FREE_LIMIT_PER_M   = 3_000_000  # бесплатные ATM/P2P/CARD по миру до 3 млн/мес

# допущения по комиссиям без премиальной (для экономии)
ATM_FEE = 0.015
P2P_FEE = 0.005
CARD_FEE = 0.005

def premium_tier(balance):
    if balance >= 6_000_000: return 0.04
    if balance >= 1_000_000: return 0.03
    return 0.02

def sum_by_month(df, mask):
    if df.empty: return pd.Series(dtype=float)
    d = df.loc[mask].copy()
    if d.empty: return pd.Series(dtype=float)
    d["ym"] = d["date"].dt.to_period("M")
    return d.groupby("ym")["amount_kzt"].sum()

def clip_sum_per_month(series, cap):
    if series.empty: return 0.0
    return float(series.clip(upper=cap).sum())

def teacher_benefits_v3(profile_row, df_tx, df_tr, thr):
    avg_balance = float(profile_row.get("avg_monthly_balance_kzt", profile_row.get("avg_balance", 0.0)) or 0.0)

    # --- Покупки, категории и месяцы
    spend_cat = {} if df_tx.empty else df_tx.groupby("category")["amount_kzt"].sum().to_dict()
    total_spend = sum(spend_cat.values())
    top3 = [k for k,_ in sorted(spend_cat.items(), key=lambda kv: kv[1], reverse=True)[:3]]
    online_sum = sum(spend_cat.get(c,0.0) for c in ONLINE)
    travel_sum = sum(spend_cat.get(c,0.0) for c in TRAVEL)

    # помесячно
    top3_mask  = df_tx["category"].isin(top3) if not df_tx.empty else pd.Series([], dtype=bool)
    online_m   = sum_by_month(df_tx, df_tx["category"].isin(ONLINE) if not df_tx.empty else pd.Series([], dtype=bool))
    top3_only_m= sum_by_month(df_tx, top3_mask & ~df_tx["category"].isin(ONLINE) if not df_tx.empty else pd.Series([], dtype=bool))
    travel_m   = sum_by_month(df_tx, df_tx["category"].isin(TRAVEL) if not df_tx.empty else pd.Series([], dtype=bool))

    # FX
    total_kzt = df_tx["amount_kzt"].sum() if not df_tx.empty else 0.0
    fx_amt = df_tx.loc[df_tx["currency"].str.upper().ne("KZT"), "amount_kzt"].sum() if not df_tx.empty else 0.0
    fx_share = (fx_amt/total_kzt) if total_kzt>0 else 0.0

    # --- ПРЕМИАЛЬНАЯ
    tier = premium_tier(avg_balance)
    base_cb = tier * total_spend
    inc_cb  = max(0.0, 0.04 - tier) * sum(spend_cat.get(c,0.0) for c in PREMIUM_4)
    # кап кешбэка помесячно
    base_inc_m = sum_by_month(df_tx, ~df_tx["category"].isna()) if not df_tx.empty else pd.Series(dtype=float)
    cashback_cap_saving = min(PREMIUM_CASHBACK_CAP_PER_M * max(1, len(base_inc_m)), base_cb + inc_cb)
    # экономия комиссий (с общим лимитом 3 млн/мес на ATM+P2P+CARD)
    saved_fees = 0.0
    if not df_tr.empty:
        df2 = df_tr.copy()
        df2["ym"] = df2["date"].dt.to_period("M")
        for ym, grp in df2.groupby("ym"):
            atm  = grp.loc[grp["type"].eq("atm_withdrawal"), "amount_kzt"].sum()
            p2p  = grp.loc[grp["type"].eq("p2p_out"), "amount_kzt"].sum()
            card = grp.loc[grp["type"].eq("card_out"), "amount_kzt"].sum()
            total_eligible = atm + p2p + card
            if total_eligible <= 0: 
                continue
            cap = PREMIUM_FREE_LIMIT_PER_M
            # распределим кап пропорционально
            scale = min(1.0, cap / total_eligible)
            saved_fees += (atm*scale*ATM_FEE) + (p2p*scale*P2P_FEE) + (card*scale*CARD_FEE)
    b_premium = cashback_cap_saving + saved_fees

    # ГЕЙТ для премиальной: нужен крупный остаток И/ИЛИ большие переводы/снятия
    out_sum = df_tr.loc[df_tr["direction"].eq("out"), "amount_kzt"].sum() if not df_tr.empty else 0.0
    premium_applicable = (avg_balance >= 1_000_000) or (out_sum >= thr["prem_out_sum"])
    if not premium_applicable:
        b_premium *= 0.25  # мягко штрафуем, не запрещаем полностью

    # --- КРЕДИТНАЯ (10% с капами, и только если есть «кредитное/онлайн» поведение)
    b_cc_top3   = 0.10 * clip_sum_per_month(top3_only_m, CC_TOP3_CAP_SUM)
    b_cc_online = 0.10 * clip_sum_per_month(online_m,   CC_ONLINE_CAP_SUM)
    b_credit = b_cc_top3 + b_cc_online

    # ГЕЙТ для кредитки
    has_cc_behavior = (not df_tr.empty) and (df_tr["type"].isin(["installment_payment_out","cc_repayment_out"]).any())
    strong_online   = (online_sum >= totals_by_client.get(profile_row["client_code"], 0)*thr["online_share"]) if totals_by_client.get(profile_row["client_code"], 0)>0 else False
    strong_topcat   = (len(top3)>0 and max(spend_cat.get(c,0.0) for c in top3)/max(total_spend,1) >= 0.30)
    cc_applicable = has_cc_behavior or strong_online or strong_topcat
    if not cc_applicable:
        b_credit *= 0.30

    # --- ТРЕВЕЛ
    b_travel = 0.04 * travel_sum
    travel_applicable = (travel_sum >= thr["travel_sum"]) or ((total_spend>0) and (travel_sum/total_spend >= thr["travel_share"]))
    if not travel_applicable:
        b_travel *= 0.20

    # --- FX
    b_fx = 0.01 * fx_amt
    if fx_share < thr["fx_share"]:
        b_fx *= 0.20

    # --- ДЕПОЗИТЫ (3 мес на свободный остаток)
    # медианная месячная трата
    if not df_tx.empty:
        m = df_tx.assign(ym=df_tx["date"].dt.to_period("M")).groupby("ym")["amount_kzt"].sum()
        med_spend = float(m.median()) if len(m) else 0.0
        vol = float(m.std() or 0.0)
    else:
        med_spend = 0.0; vol = 0.0
    free_bal = max(0.0, avg_balance - med_spend)
    months = 3/12
    dep_sav   = 0.165 * free_bal * months
    dep_nak   = 0.155 * free_bal * months
    dep_multi = 0.145 * free_bal * months if (fx_share >= thr["fx_share"]) else 0.0
    if free_bal < thr["free_balance"]:
        dep_sav *= 0.25; dep_nak *= 0.25; dep_multi *= 0.25

    # --- КРЕДИТ НАЛИЧНЫМИ (только при явной потребности)
    in_sum  = df_tr.loc[df_tr["direction"].eq("in"), "amount_kzt"].sum() if not df_tr.empty else 0.0
    net_flow = in_sum - out_sum
    b_cash = 0.0
    if (avg_balance < 100_000) and (net_flow < -300_000):
        b_cash = 5_000.0

    benefits = {
        "Карта для путешествий": b_travel,
        "Премиальная карта": b_premium,
        "Кредитная карта": b_credit,
        "Обмен валют": b_fx,
        "Депозит Сберегательный": dep_sav,
        "Депозит Накопительный": dep_nak,
        "Депозит Мультивалютный": dep_multi,
        "Инвестиции": 0.001 * max(0.0, avg_balance),
        "Золотые слитки": 0.0,
        "Кредит наличными": b_cash,
    }
    top = sorted(benefits.items(), key=lambda kv: kv[1], reverse=True)
    ctx = {"saved_fees": saved_fees, "free_bal": free_bal, "fx_amt": fx_amt, "travel_sum": travel_sum,
           "online_sum": online_sum, "top3": top3, "credit_parts": (b_cc_top3, b_cc_online)}
    return benefits, top, ctx



In [ ]:
TRAVEL = {"Путешествия","Отели","Такси"}
ONLINE = {"Едим дома","Смотрим дома","Играем дома"}
PREMIUM_4 = {"Ювелирные украшения","Косметика и Парфюмерия","Кафе и рестораны"}

# гипотезы капов (на месяц) — можно двинуть числа
CC_TOP3_CAP_SUM   = 300_000     # до этой суммы в месяц 10% на топ-3
CC_ONLINE_CAP_SUM = 100_000     # до этой суммы в месяц 10% на онлайн-сервисы
PREMIUM_CASHBACK_CAP_PER_M = 100_000  # из ТЗ (кешбэк 100k/мес)
PREMIUM_FREE_LIMIT_PER_M   = 3_000_000  # бесплатные ATM/P2P/CARD по миру до 3 млн/мес

# допущения по комиссиям без премиальной (для экономии)
ATM_FEE = 0.015
P2P_FEE = 0.005
CARD_FEE = 0.005

def premium_tier(balance):
    if balance >= 6_000_000: return 0.04
    if balance >= 1_000_000: return 0.03
    return 0.02

def sum_by_month(df, mask):
    if df.empty: return pd.Series(dtype=float)
    d = df.loc[mask].copy()
    if d.empty: return pd.Series(dtype=float)
    d["ym"] = d["date"].dt.to_period("M")
    return d.groupby("ym")["amount_kzt"].sum()

def clip_sum_per_month(series, cap):
    if series.empty: return 0.0
    return float(series.clip(upper=cap).sum())

def teacher_benefits_v3(profile_row, df_tx, df_tr, thr):
    avg_balance = float(profile_row.get("avg_monthly_balance_kzt", profile_row.get("avg_balance", 0.0)) or 0.0)

    # --- Покупки, категории и месяцы
    spend_cat = {} if df_tx.empty else df_tx.groupby("category")["amount_kzt"].sum().to_dict()
    total_spend = sum(spend_cat.values())
    top3 = [k for k,_ in sorted(spend_cat.items(), key=lambda kv: kv[1], reverse=True)[:3]]
    online_sum = sum(spend_cat.get(c,0.0) for c in ONLINE)
    travel_sum = sum(spend_cat.get(c,0.0) for c in TRAVEL)

    # помесячно
    top3_mask  = df_tx["category"].isin(top3) if not df_tx.empty else pd.Series([], dtype=bool)
    online_m   = sum_by_month(df_tx, df_tx["category"].isin(ONLINE) if not df_tx.empty else pd.Series([], dtype=bool))
    top3_only_m= sum_by_month(df_tx, top3_mask & ~df_tx["category"].isin(ONLINE) if not df_tx.empty else pd.Series([], dtype=bool))
    travel_m   = sum_by_month(df_tx, df_tx["category"].isin(TRAVEL) if not df_tx.empty else pd.Series([], dtype=bool))

    # FX
    total_kzt = df_tx["amount_kzt"].sum() if not df_tx.empty else 0.0
    fx_amt = df_tx.loc[df_tx["currency"].str.upper().ne("KZT"), "amount_kzt"].sum() if not df_tx.empty else 0.0
    fx_share = (fx_amt/total_kzt) if total_kzt>0 else 0.0

    # --- ПРЕМИАЛЬНАЯ
    tier = premium_tier(avg_balance)
    base_cb = tier * total_spend
    inc_cb  = max(0.0, 0.04 - tier) * sum(spend_cat.get(c,0.0) for c in PREMIUM_4)
    # кап кешбэка помесячно
    base_inc_m = sum_by_month(df_tx, ~df_tx["category"].isna()) if not df_tx.empty else pd.Series(dtype=float)
    cashback_cap_saving = min(PREMIUM_CASHBACK_CAP_PER_M * max(1, len(base_inc_m)), base_cb + inc_cb)
    # экономия комиссий (с общим лимитом 3 млн/мес на ATM+P2P+CARD)
    saved_fees = 0.0
    if not df_tr.empty:
        df2 = df_tr.copy()
        df2["ym"] = df2["date"].dt.to_period("M")
        for ym, grp in df2.groupby("ym"):
            atm  = grp.loc[grp["type"].eq("atm_withdrawal"), "amount_kzt"].sum()
            p2p  = grp.loc[grp["type"].eq("p2p_out"), "amount_kzt"].sum()
            card = grp.loc[grp["type"].eq("card_out"), "amount_kzt"].sum()
            total_eligible = atm + p2p + card
            if total_eligible <= 0: 
                continue
            cap = PREMIUM_FREE_LIMIT_PER_M
            # распределим кап пропорционально
            scale = min(1.0, cap / total_eligible)
            saved_fees += (atm*scale*ATM_FEE) + (p2p*scale*P2P_FEE) + (card*scale*CARD_FEE)
    b_premium = cashback_cap_saving + saved_fees

    # ГЕЙТ для премиальной: нужен крупный остаток И/ИЛИ большие переводы/снятия
    out_sum = df_tr.loc[df_tr["direction"].eq("out"), "amount_kzt"].sum() if not df_tr.empty else 0.0
    premium_applicable = (avg_balance >= 1_000_000) or (out_sum >= thr["prem_out_sum"])
    if not premium_applicable:
        b_premium *= 0.25  # мягко штрафуем, не запрещаем полностью

    # --- КРЕДИТНАЯ (10% с капами, и только если есть «кредитное/онлайн» поведение)
    b_cc_top3   = 0.10 * clip_sum_per_month(top3_only_m, CC_TOP3_CAP_SUM)
    b_cc_online = 0.10 * clip_sum_per_month(online_m,   CC_ONLINE_CAP_SUM)
    b_credit = b_cc_top3 + b_cc_online

    # ГЕЙТ для кредитки
    has_cc_behavior = (not df_tr.empty) and (df_tr["type"].isin(["installment_payment_out","cc_repayment_out"]).any())
    strong_online   = (online_sum >= totals_by_client.get(profile_row["client_code"], 0)*thr["online_share"]) if totals_by_client.get(profile_row["client_code"], 0)>0 else False
    strong_topcat   = (len(top3)>0 and max(spend_cat.get(c,0.0) for c in top3)/max(total_spend,1) >= 0.30)
    cc_applicable = has_cc_behavior or strong_online or strong_topcat
    if not cc_applicable:
        b_credit *= 0.30

    # --- ТРЕВЕЛ
    b_travel = 0.04 * travel_sum
    travel_applicable = (travel_sum >= thr["travel_sum"]) or ((total_spend>0) and (travel_sum/total_spend >= thr["travel_share"]))
    if not travel_applicable:
        b_travel *= 0.20

    # --- FX
    b_fx = 0.01 * fx_amt

    # --- ДЕПОЗИТЫ (3 мес на свободный остаток)
    # медианная месячная трата
    if not df_tx.empty:
        m = df_tx.assign(ym=df_tx["date"].dt.to_period("M")).groupby("ym")["amount_kzt"].sum()
        med_spend = float(m.median()) if len(m) else 0.0
        vol = float(m.std() or 0.0)
    else:
        med_spend = 0.0; vol = 0.0
    free_bal = max(0.0, avg_balance - med_spend)
    months = 3/12
    dep_sav   = 0.165 * free_bal * months
    dep_nak   = 0.155 * free_bal * months
    dep_multi = 0.145 * free_bal * months
    if free_bal < thr["free_balance"]:
        dep_sav *= 0.25; dep_nak *= 0.25; dep_multi *= 0.25

    # --- КРЕДИТ НАЛИЧНЫМИ (только при явной потребности)
    in_sum  = df_tr.loc[df_tr["direction"].eq("in"), "amount_kzt"].sum() if not df_tr.empty else 0.0
    net_flow = in_sum - out_sum
    b_cash = 0.0
    if (avg_balance < 100_000) and (net_flow < -300_000):
        b_cash = 5_000.0

    benefits = {
        "Карта для путешествий": b_travel,
        "Премиальная карта": b_premium,
        "Кредитная карта": b_credit,
        "Обмен валют": b_fx,
        "Депозит Сберегательный": dep_sav,
        "Депозит Накопительный": dep_nak,
        "Депозит Мультивалютный": dep_multi,
        "Инвестиции": 0.001 * max(0.0, avg_balance),
        "Золотые слитки": 0.0,
        "Кредит наличными": b_cash,
    }
    top = sorted(benefits.items(), key=lambda kv: kv[1], reverse=True)
    ctx = {"saved_fees": saved_fees, "free_bal": free_bal, "fx_amt": fx_amt, "travel_sum": travel_sum,
           "online_sum": online_sum, "top3": top3, "credit_parts": (b_cc_top3, b_cc_online)}
    return benefits, top, ctx


In [ ]:
MONTHS_GEN = {1:"январе",2:"феврале",3:"марте",4:"апреле",5:"мае",6:"июне",7:"июле",8:"августе",9:"сентябре",10:"октябре",11:"ноябре",12:"декабре"}

def kzt(amount, decimals=0):
    a = float(amount)
    if decimals==0:
        s = f"{int(round(a)):,}".replace(",", " ")
        return f"{s} ₸"
    s = f"{a:,.{decimals}f}".replace(",", " ").replace(".", ",")
    return f"{s} ₸"

def trim_push(s, limit=220):
    s = " ".join(str(s).split())
    return (s[:limit].rstrip(" .,;") + "…") if len(s)>limit else s

def month_of_max(df, cats):
    if df.empty: return "последние месяцы"
    d = df[df["category"].isin(cats)]
    if d.empty: return "последние месяцы"
    d = d.assign(ym=d["date"].dt.to_period("M"))
    s = d.groupby("ym")["amount_kzt"].sum()
    ym = s.idxmax()
    try:
        return MONTHS_GEN[int(str(ym)[5:7])]
    except Exception:
        return "последние месяцы"

def top3_with_amounts(spend_cat):
    items = sorted(spend_cat.items(), key=lambda kv: kv[1], reverse=True)[:3]
    return [f"{k} ({kzt(v)})" for k,v in items]

def gen_push_v3(name, product, profile_row, df_tx, df_tr, ctx):
    spend_cat = {} if df_tx.empty else df_tx.groupby("category")["amount_kzt"].sum().to_dict()
    if product == "Карта для путешествий":
        m = month_of_max(df_tx, TRAVEL)
        benefit = 0.04 * sum(spend_cat.get(c,0.0) for c in TRAVEL)
        return trim_push(f"{name}, в {m} у вас много поездок и такси. С тревел-картой вернулось бы ≈{kzt(benefit)} за 3 мес. Открыть карту.")
    if product == "Премиальная карта":
        saved = ctx.get("saved_fees", 0.0)
        return trim_push(f"{name}, у вас стабильный остаток и частые переводы/снятия. Премиальная карта даст до 4% и сэкономит комиссии ≈{kzt(saved)}. Оформить сейчас.")
    if product == "Кредитная карта":
        t3 = ", ".join([c.split()[0] for c in top3_with_amounts(spend_cat)]) if spend_cat else "ваши покупки"
        b_top3, b_online = ctx.get("credit_parts",(0,0))
        return trim_push(f"{name}, топ-категории — {t3}. Кредитная карта даст до 10% (≈{kzt(b_top3+b_online)}) и рассрочку 3–24 мес без переплат. Оформить карту.")
    if product == "Обмен валют":
        fx_amt = ctx.get("fx_amt", 0.0)
        return trim_push(f"{name}, за 3 мес тратили в валюте на ≈{kzt(fx_amt)}. В приложении выгодный обмен и авто-покупка по целевому курсу. Настроить обмен.")
    if product in {"Депозит Сберегательный","Депозит Накопительный","Депозит Мультивалютный"}:
        free_bal = ctx.get("free_bal", 0.0)
        rate = {"Депозит Сберегательный":0.165,"Депозит Накопительный":0.155,"Депозит Мультивалютный":0.145}[product]
        income = rate * max(0.0, free_bal) * (3/12)
        return trim_push(f"{name}, свободно лежит ≈{kzt(free_bal)}. На «{product}» за 3 мес получили бы ≈{kzt(income)}. Открыть вклад.")
    if product == "Инвестиции":
        return trim_push(f"{name}, попробуйте инвестиции с низким порогом входа и без комиссий на старт. Открыть счёт.")
    if product == "Кредит наличными":
        return trim_push(f"{name}, если нужен запас на крупные траты — оформите кредит наличными с гибкими выплатами. Узнать лимит.")
    return trim_push(f"{name}, у вас есть возможность получить выгоду с нашим продуктом. Посмотреть детали. Открыть.")


In [ ]:
tx_groups = dict(tuple(tx.groupby("client_code"))) if not tx.empty else {}
tr_groups = dict(tuple(tr.groupby("client_code"))) if not tr.empty else {}

rows = []
for _, p in clients.iterrows():
    code = str(p["client_code"]); name = str(p["name"])
    df_tx = tx_groups.get(code, pd.DataFrame(columns=tx.columns))
    df_tr = tr_groups.get(code, pd.DataFrame(columns=tr.columns))

    ben, top, ctx = teacher_benefits_v3(p, df_tx, df_tr, thr)
    product = top[0][0]
    push = gen_push_v3(name, product, p, df_tx, df_tr, ctx)
    rows.append({"client_code": code, "product": product, "push_notification": push})

result = pd.DataFrame(rows)
display(result.head(10))
print(result["product"].value_counts())
result.to_csv("outputs/result.csv", index=False, encoding="utf-8")
print("Saved: outputs/result.csv")
